In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [ ]:
import pickle
import pandas as pd
import numpy as np
import sys 

import os


In [4]:



with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/mlops/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mlops/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [21]:
year = 2022
month = 2
taxi_type = 'yellow'

taxi_type = sys.argv[1]  # 'yellow'
year = int(sys.argv[2])       #2022
month = int(sys.argv[3])      #2


df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet')



In [22]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [25]:


# Calculate the standard deviation of the predicted duration
std_dev = np.std(y_pred)

# Print the standard deviation
print("Standard Deviation of Predicted Duration:", std_dev)




Standard Deviation of Predicted Duration: 5.28140357655334


In [29]:
#!mkdir output

In [33]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')



output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'
df_result = df[['ride_id', 'duration']]
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [37]:
# Get the size of the output file
file_size = os.path.getsize(output_file)

# Convert the size to human-readable format
size_in_mb = file_size / (1024 * 1024)

# Print the size of the output file
print("Size of the output file:", size_in_mb, "MB")

Size of the output file: 56.169318199157715 MB
